In [2]:
import os
import sys

projdir = os.path.abspath(os.path.join(os.path.abspath(''), '..'))
sys.path.insert(1, projdir)
os.chdir(projdir)

# 이 셀은 한번만 돌려야함!!
projdir

'/home/maxjo/Projects/ds_pbl'

In [3]:
from utils.DB_connect import get_engine
engine = get_engine()

from glob import glob

import pandas as pd
from sqlalchemy import text
from sqlalchemy.types import Date, String

---
delays

In [28]:
file_pattern = os.path.join('data', 'subway_data', '노선별_지연시간_*')
files = glob(file_pattern)
print("files:", files)

df_subway_delay = pd.concat([pd.read_csv(file, encoding="euc-kr") for file in files], ignore_index=True)
df_subway_delay.drop(columns=["연번", "번호"], inplace=True)

df_subway_delay['방향'] = df_subway_delay['노선'].apply(lambda x: x.split()[1][1:-1])
df_subway_delay['노선'] = df_subway_delay['노선'].apply(lambda x: x.split()[0])

# 평일, 토요일, 일요일로 구분
df_subway_delay['지연일자'] = pd.to_datetime(df_subway_delay['지연일자'], format='%Y-%m-%d')
df_subway_delay['dow'] = df_subway_delay['지연일자'].dt.dayofweek
df_subway_delay['요일구분'] = df_subway_delay['dow'].map(
    lambda x: '평일'   if x < 5 else
              '토요일' if x == 5 else
              '일요일'
)
ni = ['요일구분', '노선', '지연시간대', '최대지연시간', '방향', 'dow', '지연일자']
df_subway_delay = df_subway_delay[ni].drop(columns=['dow'])


df_subway_delay

files: ['data/subway_data/노선별_지연시간_20230831.csv', 'data/subway_data/노선별_지연시간_20241004.csv']


,요일구분,노선,지연시간대,최대지연시간,방향,지연일자
0,일요일,7호선,09시~18시,5분,부천구청행,2023-01-01
1,일요일,7호선,09시~18시,5분,온수행,2023-01-01
2,일요일,7호선,09시~18시,5분,장암행,2023-01-01
3,평일,2호선,첫차~09시,20분,외선,2023-01-02
4,평일,7호선,18시~막차,5분,부천구청행,2023-01-02
...,...,...,...,...,...,...
3266,평일,4호선,첫차~09시,10분,하행선,2024-08-30
3267,평일,4호선,첫차~09시,10분,상행선,2024-08-30
3268,평일,3호선,첫차~09시,10분,하행선,2024-08-30
3269,평일,3호선,첫차~09시,5분,상행선,2024-08-30


In [29]:
df_subway_delay.to_sql(
    name='subway_delays',
    con=engine,
    if_exists='replace',  # Options: 'fail', 'replace', 'append'
    index=False,
    dtype={
        '지연일자': Date(),
        '노선': String(50),
        '지연시간대': String(10),
        '최대지연시간': String(10),
    } # type: ignore
)

3271

---
Sta. congestion (사용 안하게 됨)

In [6]:
file_pattern = os.path.join('data', 'subway_data', 'subway_sta_*')
files = glob(file_pattern)
print("files:", files)

files_df = list()
for file in files:  # 인코딩들이이 섞여있음
    df = None
    try:
        df = pd.read_csv(file, encoding="utf-8-sig", index_col=False)
        print("Reading as utf-8-sig", end=': ')
        # print(df.columns)
        
    except UnicodeDecodeError:
        df = pd.read_csv(file, encoding="euc-kr", index_col=False)
        print("Reading as euc-kr", end=': ')
        # print(df.columns)
    
    print(len(df))
    files_df.append(df)

df_station_users = pd.concat(files_df)

df_station_users.drop(columns=["등록일자"], inplace=True)
df_station_users.rename(columns={'사용일자': '일자', '노선명': '노선'}, inplace=True)

# 날짜 포멧으로 변경
df_station_users['일자'] = df_station_users['일자'].astype(str)
df_station_users['일자'] = df_station_users['일자'].apply(lambda x: f'{x[:4]}-{x[4:6]}-{x[6:8]}')

df_station_users

files: ['data/subway_data/subway_sta_202406.csv', 'data/subway_data/subway_sta_202309.csv', 'data/subway_data/subway_sta_202404.csv', 'data/subway_data/subway_sta_202407.csv', 'data/subway_data/subway_sta_202408.csv', 'data/subway_data/subway_sta_202405.csv', 'data/subway_data/subway_sta_202311.csv', 'data/subway_data/subway_sta_202312.csv', 'data/subway_data/subway_sta_202401.csv', 'data/subway_data/subway_sta_202402.csv', 'data/subway_data/subway_sta_202403.csv', 'data/subway_data/subway_sta_202310.csv']
Reading as utf-8-sig: 18488
Reading as utf-8-sig: 18302
Reading as utf-8-sig: 18483
Reading as utf-8-sig: 19103
Reading as utf-8-sig: 19132
Reading as utf-8-sig: 19110
Reading as utf-8-sig: 18313
Reading as utf-8-sig: 18956
Reading as utf-8-sig: 19001
Reading as euc-kr: 17807
Reading as utf-8-sig: 19079
Reading as utf-8-sig: 18911


,일자,노선,역명,승차총승객수,하차총승객수
0,2024-06-01,경부선,신도림,3746,4163
1,2024-06-01,3호선,신사,24723,26535
2,2024-06-01,3호선,압구정,26592,28521
3,2024-06-01,3호선,옥수,8041,8143
4,2024-06-01,3호선,금호,6886,6157
...,...,...,...,...,...
18906,2023-10-31,중앙선,오빈,356,370
18907,2023-10-31,중앙선,양평,3385,3385
18908,2023-10-31,중앙선,원덕,405,374
18909,2023-10-31,중앙선,용문,2877,2785


In [7]:
df_station_users.to_sql(
    name='station_users',
    con=engine,
    if_exists='replace',  # Options: 'fail', 'replace', 'append'
    index=False,
    dtype={
        '일자': Date(),
        '노선': String(10),
        '역명': String(50),
    } # type: ignore
)

224685

---
Train congestion

서울교통공사 1-8호선 30분 단위 평균 혼잡도로 30분간 지나는 열차들의 평균 혼잡도(정원대비 승차인원으로, 승차인과 좌석수가 일치할 경우를 혼잡도 34%로 산정) 입니다.(단위: %). 서울교통공사 혼잡도 데이터는 요일구분(평일, 토요일, 일요일), 호선, 역번호, 역명, 상하선구분, 30분단위 별 혼잡도 데이터로 구성되어 있습니다. (2024년 부터 분기별 데이터가 제공됩니다.) 

In [ ]:
file_pattern = os.path.join('data', 'subway_data', '지하철*')
files = glob(file_pattern)
print("files:", files)

files_df = list()
for file in files:
    df = None
    try:
        df = pd.read_csv(file, encoding="utf-8-sig", index_col=False)
        print("Reading as utf-8-sig", end=': ')
        # print(df.columns)
        
    except UnicodeDecodeError:
        df = pd.read_csv(file, encoding="euc-kr", index_col=False)
        print("Reading as euc-kr", end=': ')
        # print(df.columns)

    df.drop(columns=["연번"], inplace=True, errors='ignore')  # 연번이 있는경우도 없는경우도 있음
    oc = list(df.columns)  # 원래 컬럼명 저장, 이후 컬럼 reorder용

    date = file[-12:-4]  # 파일명에서 일자 추출
    df['일자'] = f'{date[:4]}-{date[4:6]}-{date[6:8]}'  # YYYY-MM-DD 포멧으로 변경

    df = df[['일자'] + oc]  # 일자 컬럼을 맨 앞으로 이동

    print(len(df))
    files_df.append(df)

df_train_congestions = pd.concat(files_df)

df_train_congestions.drop(columns=["역번호", ], inplace=True)
df_train_congestions.rename(columns={'호선': '노선'}, inplace=True)

df_train_congestions['노선'] = df_train_congestions['노선'].astype(str)
df_train_congestions['노선'] = df_train_congestions['노선'].apply(lambda x: f'{x}호선')

# 빈공간이 있는 요일들 있음 - select distinct(요일구분) from train_congestions;으로 검증
df_train_congestions['요일구분'] = df_train_congestions['요일구분'].apply(lambda x: x.strip())

df_train_congestions

files: ['data/subway_data/지하철혼잡도정보_20240930.csv', 'data/subway_data/지하철혼잡도정보_20240630.csv', 'data/subway_data/지하철혼잡도정보_20240331.csv', 'data/subway_data/지하철혼잡도정보_20241231.csv', 'data/subway_data/지하철혼잡도정보_20231231.csv']
Reading as euc-kr: 1674
Reading as euc-kr: 1668
Reading as euc-kr: 1668
Reading as euc-kr: 1662
Reading as euc-kr: 1668


,일자,요일구분,노선,출발역,상하구분,5시30분,6시00분,6시30분,7시00분,7시30분,...,20시00분,20시30분,21시00분,21시30분,22시00분,22시30분,23시00분,23시30분,00시00분,00시30분
0,2024-09-30,평일,1호선,청량리,상선,9.7,8.7,5.0,8.7,10.3,...,26.1,25.7,27.8,23.8,22.4,23.7,22.9,15.1,8.7,0.0
1,2024-09-30,평일,1호선,청량리,하선,21.7,19.2,29.2,45.6,70.2,...,11.3,9.8,11.4,10.3,8.2,9.4,8.2,8.5,2.4,0.2
2,2024-09-30,평일,1호선,제기동,상선,10.0,10.7,7.5,9.1,12.5,...,30.5,25.5,34.3,27.0,25.0,26.9,25.6,17.2,11.9,0.0
3,2024-09-30,평일,1호선,제기동,하선,21.7,20.4,30.2,45.6,63.2,...,13.3,11.0,11.7,10.5,9.1,10.0,8.5,8.6,2.3,0.3
4,2024-09-30,평일,1호선,신설동,상선,9.0,13.3,8.2,10.1,14.5,...,30.9,26.2,35.4,27.9,26.0,28.1,25.4,16.6,13.1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1663,2023-12-31,일요일,8호선,신흥,하선,7.1,3.2,3.8,2.5,5.4,...,3.7,3.5,4.5,3.6,3.0,2.6,2.6,1.5,0.8,0.0
1664,2023-12-31,일요일,8호선,수진,상선,2.0,1.8,2.9,2.7,2.4,...,5.2,5.2,6.3,5.6,3.6,4.9,2.9,2.8,0.0,0.0
1665,2023-12-31,일요일,8호선,수진,하선,9.8,4.3,3.4,2.7,4.6,...,2.3,1.9,3.0,1.8,1.6,1.1,1.0,0.7,0.3,0.0
1666,2023-12-31,일요일,8호선,모란,상선,1.2,0.8,2.5,2.1,1.9,...,5.9,5.7,6.5,6.2,3.7,5.0,3.4,2.9,0.0,0.0


In [13]:
df_train_congestions.to_sql(
    name='train_congestions',
    con=engine,
    if_exists='replace',  # Options: 'fail', 'replace', 'append'
    index=False,
    dtype={
        '일자': Date(),
    } # type: ignore
)

8340